In [1]:
# Uncomment the following code if you are using Colab.
!pip install gradio

In [2]:
#Import libraries
import gradio as gr
import numpy as np
import pandas as pd
from PIL import Image
from keras.models import load_model
from keras.preprocessing import image

In [3]:
# Load the trained Keras model
model = load_model('warbler_model.keras', compile=False)

In [105]:
# read in fun facts into a df
fun_df = pd.read_csv("Fun Facts about Warblers.csv", usecols=["Species Name", "Fun Fact 3"])

# Define a function to process the image and make predictions
def predict_bird_species(img):
    # Preprocess the image for the model
    img = img.resize((224, 224))  # Resize to the input size of the model
    img_array = image.img_to_array(img)  # Convert image to array
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Normalize the image

    # Make predictions
    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions, axis=1)  # Get the class with the highest probability

    # Map the predicted class to the species name
    species_names = ["Bay_breasted_Warbler", "Black_and_white_Warbler", "Black_throated_Blue_Warbler", "Blue_winged_Warbler", "Canada_Warbler", \
                     "Cape_May_Warbler", "Cerulean_Warbler", "Chestnut_sided_Warbler", "Golden_winged_Warbler", "Hooded_Warbler", "Kentucky_Warbler", \
                     "Magnolia_Warbler", "Mourning_Warbler", "Myrtle_Warbler", "Nashville_Warbler", "Orange_crowned_Warbler", "Palm_Warbler", "Pine_Warbler",\
                     "Prairie_Warbler", "Prothonotary_Warbler", "Swainson_Warbler", "Tennessee_Warbler", "Wilson_Warbler", "Worm_eating_Warbler", \
                     "Yellow_Warbler"]
    fun_fact = fun_df.loc[fun_df["Species Name"] == species_names[predicted_class[0]], "Fun Fact 3"].values[0][3:]
    bird_image_path = f"single_image_warblers/{species_names[predicted_class[0]]}.jpg"
    bird_image = Image.open(bird_image_path)
    return species_names[predicted_class[0]], bird_image, fun_fact  # Return species name and the original image

In [106]:
# Create the Gradio interface
iface = gr.Interface(
    fn=predict_bird_species,
    inputs=gr.Image(type="pil"),  # Input component for image upload
    outputs=[gr.Label(), gr.Image(type="pil"), gr.Label()],  # Output for species name and the image
    title="Bird Species Prediction",
    description="Upload a picture of a bird to predict its species."
)

In [109]:
# Launch the app
iface.launch(share=True)

Rerunning server... use `close()` to stop if you need to change `launch()` parameters.
----
* Running on public URL: https://c2e1f33fcec6c1f36c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
